In [ ]:
#На данном этапе необходимо иметь готовый X_train, Y_train 
data_train = lgb.Dataset(X_train, label=Y_train)
rkf = StratifiedKFold(n_splits=4, random_state=48, shuffle = True)

#training rounds и best score понадобится чтобы определить оптимальное число num_boost_round
best_score = 999
training_rounds = 10000

def objective(trial):
    global best_score, training_rounds
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_classes": 7,
        "seed": 48,
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 120),
        'feature_pre_filter': False
    }
    
    lgbcv = lgb.cv(param,
                   data_train,
                   folds=rkf,
                   verbose_eval=False,
                   categorical_feature=cat_features,
                   early_stopping_rounds=300,                   
                   num_boost_round=15000,                    
                   callbacks=[lgb.reset_parameter(learning_rate = [0.005]*300 + [0.001]*7300 + [0.0005]*7400) ]
                  )
    
    cv_score = lgbcv['multi_logloss-mean'][-1]
    #Если cv_score является лучшим, то запоминаем число training_rounds
    if cv_score<best_score:
        training_rounds = len( list(lgbcv.values())[0] )
        best_score = cv_score
    
    return cv_score

study = optuna.create_study(direction='minimize')  
study.optimize(objective, timeout=27000) 